In [1]:
"""
 * @ Author: Lucas Glasner (lgvivanco96@gmail.com)
 * @ Create Time: 1969-12-31 21:00:00
 * @ Modified by: Lucas Glasner, 
 * @ Modified time: 2024-09-27 10:48:25
 * @ Description:
        Python translation of the "rational method" for computing peak runoff
        on multiple drainage basins. 
 * @ Dependencies:
 */

"""
import pandas as pd
import numpy as np
import os

from hydrocivil.geomorphology import concentration_time
from hydrocivil.rain import duration_coef

In [27]:
def multiple_tc(parameters):
    """
    Given a dataframe with geomorpholical properties this function returns
    the concentration time computed with different formulas

    Args:
        parameters (pandas.DataFrame): Table with basin parameters

    Returns:
        (pandas.DataFrame): Concentration time in minutes
    """
    tc_california = concentration_time('California',
                                       mriverlen=parameters.loc['mriverlen'],
                                       hmax=parameters.loc['hmax'],
                                       hmin=parameters.loc['hmin'])
    tc_giandotti = concentration_time('Giandotti',
                                      mriverlen=parameters.loc['mriverlen'],
                                      hmean=parameters.loc['hmean'],
                                      hmin=parameters.loc['hmin'],
                                      area=parameters.loc['area'])
    tc_kirpich = concentration_time('Kirpich',
                                    mriverlen=parameters.loc['mriverlen'],
                                    hmax=parameters.loc['hmax'],
                                    hmin=parameters.loc['hmin'])
    tc_spain = concentration_time('Spain',
                                  mriverlen=parameters.loc['mriverlen'],
                                  meanslope=parameters.loc['meanslope'])
    tc_SCS = concentration_time('SCS',
                                mriverlen=parameters.loc['mriverlen'],
                                meanslope=parameters.loc['meanslope'],
                                curvenumber=parameters.loc['curvenumber'])
    tc = pd.concat([tc_california, tc_giandotti, tc_kirpich, tc_spain, tc_SCS],
                   axis=1)
    tc.columns = ['California', 'Giandotti', 'Kirpich', 'Spain', 'SCS']
    return tc
    
def RationalMethod(parameters, pr_mm24hr, **kwargs):
    """
    Given dataframes with basin parameters and precipitations this function
    computes the peak runoff of the rational method. 

    Args:
        parameters (DataFrame): pandas DataFrame with basin parameters
        pr_mm24hr (DataFrame): pandas DataFrame with precipitation for
            each basin and return period.

    Returns:
        basin_tcs, pr_mmXhr, peakrunoff (tuple):
            DataFrames with concentration time, precipitation intensity at the
            concentration time and model's peak runoff. 
    """
    # Runoff coefficient "frequency factors"
    runoff_FF  = pd.Series([1, 1, 1, 1.1, 1.15, 1.2,1.225, 1.25, 1.275, 1.3],
                        index=[2,5,10,20,25,50,75,100,150,200])
    runoff_FF  = runoff_FF.loc[pr_mm24hr.index]
    pr_mm24hr = pr_mm24hr.copy()
    parameters = parameters.copy()
    # Compute concentration time and change units to hours
    basin_tcs = multiple_tc(parameters)/60

    # Compute runoff coefficient for all basins and return periods
    runoff_coefs = [runoff_FF*parameters.loc['runoffcoef'].loc[basin]
                    for basin in parameters.columns]
    runoff_coefs = pd.concat(runoff_coefs, keys=parameters.columns).unstack().T
    
    # Compute the duration coefficient for the concentration time
    DCoeffs = [duration_coef(basin_tcs.iloc[:,i].values, **kwargs)
                            for i in range(len(basin_tcs.columns))]
    DCoeffs = pd.DataFrame(DCoeffs,
                           columns=basin_tcs.index,
                           index=basin_tcs.columns)
    
    # Compute rainfall intensity associated with the concentration time
    I_mmhr = pd.concat([pr_mm24hr*DCoeffs.loc[method]/basin_tcs[method]
                        for method in DCoeffs.index],
                        keys=DCoeffs.index)
    
    # Compute peak runoff
    basin_areas  = parameters.loc['area']
    peakrunoff   = [runoff_coefs*I_mmhr.loc[method]*basin_areas/3.6
                    for method in DCoeffs.index]
    peakrunoff   = pd.concat(peakrunoff, keys=DCoeffs.index)
    return basin_tcs.T, I_mmhr, peakrunoff

In [35]:
%%time
input_params = r'params.xlsx'
out = r'QRational_Summary.xlsx'
# ------------------------------ load input data ----------------------------- #
params = pd.read_excel(input_params, index_col=0, sheet_name='params')
precips_mm24hr = pd.read_excel(input_params, index_col=0, sheet_name='rain')

# ------------------------- run model for all basins ------------------------- #
basin_tcs, precips_mmXhr, peakrunoff = RationalMethod(params, precips_mm24hr,
                                                      duration_threshold=0)
        
# --------------------------- save results summary --------------------------- #
with pd.ExcelWriter(out, mode='w', engine='openpyxl') as writer:
    params.to_excel(writer, sheet_name='parameters')
with pd.ExcelWriter(out, mode='a', engine='openpyxl') as writer:
    basin_tcs.to_excel(writer, sheet_name='concentration_time_hr')
    precips_mmXhr.to_excel(writer, sheet_name='rainfall_intensity_mmhr-1')
    peakrunoff.to_excel(writer, sheet_name='peakrunoff_m3s')

CPU times: total: 46.9 ms
Wall time: 174 ms
